In [ ]:
from google.colab import drive
drive.mount('/content/drive')



!unzip -qq "drive/My Drive/tiny-imagenet-200.zip"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
from __future__ import division

import six
from keras.models import Model
from keras.layers import (
    Input,
    Activation,
    Dense,
    GlobalAveragePooling2D,
    Flatten
)
from keras.layers.convolutional import (
    Conv2D,
    Convolution2D,
    MaxPooling2D,
    AveragePooling2D
    
)
from keras.layers.merge import add
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator


from keras.callbacks import *

Using TensorFlow backend.


In [ ]:
def relu(x):
    return Activation('relu')(x)

def bn_relu_conv(f1,f2,stride):
    def unit(x):
        nBottleneckPlane = int(f2 / 4)
        nbp = nBottleneckPlane
        if f1==f2:
            ident = x

            x = BatchNormalization(axis=-1)(x)
            x = relu(x)
            x = Conv2D(nbp,(1,1),
            strides=(stride,stride))(x)

            x = BatchNormalization(axis=-1)(x)
            x = relu(x)
            x = Conv2D(nbp,(3,3),padding='same')(x)

            x = BatchNormalization(axis=-1)(x)
            x = relu(x)
            x = Conv2D(f2,(1,1))(x)

            out = add([ident, x])
        else:
            x = BatchNormalization(axis=-1)(x)
            x = relu(x)
            ident = x

            x = Conv2D(nbp,(1,1),
            strides=(stride,stride))(x)

            x = BatchNormalization(axis=-1)(x)
            x = relu(x)
            x = Conv2D(nbp,(3,3),padding='same')(x)

            x = BatchNormalization(axis=-1)(x)
            x = relu(x)
            x = Conv2D(f2,(1,1))(x)

            ident = Conv2D(f2,(1,1),
            strides=(stride,stride))(ident)

            out = add([ident, x])

        return out
    return unit      

def resblock(f1,f2,layers,std):
    def unit(x):
        for i in range(layers):
            if i==0:
                x = bn_relu_conv(f1,f2,std)(x)
            else:
                x = bn_relu_conv(f2,f2,1)(x)
        return x
    return unit      

inp = Input(shape=(32,32,3))
i = inp

i = Conv2D(64,(3,3),padding='same')(i)

i = resblock(64,128,3,1)(i) #32x32
i = resblock(128,256,3,2)(i) #16x16
i = resblock(256,512,3,2)(i) #8x8

i = BatchNormalization(axis=-1)(i)
i = relu(i)

i = AveragePooling2D(pool_size=(8,8),padding='valid')(i) #1x1

i = Conv2D(200,1)(i)
#i = Flatten()(i) # 128
i = GlobalAveragePooling2D()(i)

#i = Dense(10)(i)
i = Activation('softmax')(i)

model = Model(inputs=inp,outputs=i)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 64)   1792        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 64)   256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 64)   0           batch_normalization_1[0][0]      
_____________________________________

In [ ]:
#LOAD IMAGES

import numpy as np
from PIL import Image
np.random.seed(1337)  # for reproducibility

#Keras
from keras.models import Sequential
from keras.layers.core import Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization 
from keras.utils import np_utils
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
# from keras.utils.visualize_util import plot
import h5py

import os
import numpy as np
from PIL import Image

def get_annotations_map():
  
  valAnnotationsPath = './32tiny-imagenet-200/tiny-imagenet-200/val/val_annotations.txt'
  valAnnotationsFile = open(valAnnotationsPath, 'r')
  valAnnotationsContents = valAnnotationsFile.read()
  valAnnotations = {}
  for line in valAnnotationsContents.splitlines():
    pieces = line.strip().split()
    valAnnotations[pieces[0]] = pieces[1]
  return valAnnotations

def load_images(path,num_classes):
    #Load images
    
    print('Loading ' + str(num_classes) + ' classes')

    X_train=np.zeros([num_classes*500,3,32,32],dtype='uint8')
    y_train=np.zeros([num_classes*500], dtype='uint8')
    trainPath=path+'/train'

    print('loading training images...');

    i=0
    j=0
    annotations={}
    for sChild in os.listdir(trainPath):
        sChildPath = os.path.join(os.path.join(trainPath,sChild),'images')
        annotations[sChild]=j
        for c in os.listdir(sChildPath):
            X=np.array(Image.open(os.path.join(sChildPath,c)))
            if len(np.shape(X))==2:
                X_train[i]=np.array([X,X,X])
            else:
                X_train[i]=np.transpose(X,(2,0,1))
            y_train[i]=j
            i+=1
        j+=1
        if (j >= num_classes):
            break

    print('finished loading training images')

    val_annotations_map = get_annotations_map()

    X_test = np.zeros([num_classes*50,3,32,32],dtype='uint8')
    y_test = np.zeros([num_classes*50], dtype='uint8')


    print('loading test images...')

    i = 0
    testPath=path+'/val/images'
    for sChild in os.listdir(testPath):
        if val_annotations_map[sChild] in annotations.keys():
            sChildPath = os.path.join(testPath, sChild)
            X=np.array(Image.open(sChildPath))
            if len(np.shape(X))==2:
                X_test[i]=np.array([X,X,X])
            else:
                X_test[i]=np.transpose(X,(2,0,1))
            y_test[i]=annotations[val_annotations_map[sChild]]
            i+=1
        else:
            pass


    print('finished loading test images ',end="")
    print(i)

    return X_train,y_train,X_test,y_test

loss_functions = ['categorical_crossentropy','squared_hinge','hinge']
num_classes = 200
batch_size = 128
nb_epoch = 30

#Load images
path='./32tiny-imagenet-200/tiny-imagenet-200/'
X_train,y_train,X_test,y_test=load_images(path,num_classes)

X_train = X_train.transpose(0,3,1,2)
X_test = X_test.transpose(0,3,1,2)

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print('X_test shape:', X_test.shape)
print(X_test.shape[0], 'test samples')

num_samples=len(X_train)

# input image dimensions
num_channels , img_rows, img_cols = X_train.shape[1], X_train.shape[2], X_train.shape[3]

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, num_classes)
Y_test = np_utils.to_categorical(y_test, num_classes)

Loading 200 classes
loading training images...
finished loading training images
loading test images...
finished loading test images 10000
X_train shape: (100000, 32, 3, 32)
100000 train samples
X_test shape: (10000, 32, 3, 32)
10000 test samples


In [ ]:
X_train = X_train.transpose(0,3,1,2)
X_test = X_test.transpose(0,3,1,2)
print (X_train.shape)
print (X_test.shape)

(100000, 32, 32, 3)
(10000, 32, 32, 3)


In [ ]:
from keras.callbacks import ModelCheckpoint

filepath="./drive/My Drive/ChkPoint3/epochs:{epoch:03d}-val_acc:{val_acc:.3f}.hdf5"

checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

callbacks_list = [checkpoint]

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# W/O ImgAug

model.fit(X_train, Y_train,
          batch_size=128,
          epochs=10,
          validation_data=(X_test, Y_test),
          shuffle=True,
          callbacks=callbacks_list)

Train on 100000 samples, validate on 10000 samples
Epoch 1/10
100000/100000 [==============================] - 259s 3ms/step - loss: 4.3157 - acc: 0.0986 - val_loss: 4.4449 - val_acc: 0.0984

Epoch 00001: val_acc improved from -inf to 0.09840, saving model to ./drive/My Drive/ChkPoint3/epochs:001-val_acc:0.098.hdf5
Epoch 2/10
100000/100000 [==============================] - 251s 3ms/step - loss: 3.5479 - acc: 0.2027 - val_loss: 3.7905 - val_acc: 0.1712

Epoch 00002: val_acc improved from 0.09840 to 0.17120, saving model to ./drive/My Drive/ChkPoint3/epochs:002-val_acc:0.171.hdf5
Epoch 3/10
100000/100000 [==============================] - 250s 3ms/step - loss: 3.1862 - acc: 0.2650 - val_loss: 3.5069 - val_acc: 0.2161

Epoch 00003: val_acc improved from 0.17120 to 0.21610, saving model to ./drive/My Drive/ChkPoint3/epochs:003-val_acc:0.216.hdf5
Epoch 4/10
100000/100000 [==============================] - 250s 3ms/step - loss: 2.9317 - acc: 0.3118 - val_loss: 3.2766 - val_acc: 0.2591

Epoc

In [ ]:
def relu(x):
    return Activation('relu')(x)

def bn_relu_conv(f1,f2,stride):
    def unit(x):
        nBottleneckPlane = int(f2 / 4)
        nbp = nBottleneckPlane
        if f1==f2:
            ident = x

            x = BatchNormalization(axis=-1)(x)
            x = relu(x)
            x = Conv2D(nbp,(1,1),
            strides=(stride,stride))(x)

            x = BatchNormalization(axis=-1)(x)
            x = relu(x)
            x = Conv2D(nbp,(3,3),padding='same')(x)

            x = BatchNormalization(axis=-1)(x)
            x = relu(x)
            x = Conv2D(f2,(1,1))(x)

            out = add([ident, x])
        else:
            x = BatchNormalization(axis=-1)(x)
            x = relu(x)
            ident = x

            x = Conv2D(nbp,(1,1),
            strides=(stride,stride))(x)

            x = BatchNormalization(axis=-1)(x)
            x = relu(x)
            x = Conv2D(nbp,(3,3),padding='same')(x)

            x = BatchNormalization(axis=-1)(x)
            x = relu(x)
            x = Conv2D(f2,(1,1))(x)

            ident = Conv2D(f2,(1,1),
            strides=(stride,stride))(ident)

            out = add([ident, x])

        return out
    return unit      

def resblock(f1,f2,layers,std):
    def unit(x):
        for i in range(layers):
            if i==0:
                x = bn_relu_conv(f1,f2,std)(x)
            else:
                x = bn_relu_conv(f2,f2,1)(x)
        return x
    return unit      

inp = Input(shape=(64,64,3))
i = inp

i = Conv2D(64,(3,3),padding='same')(i)

i = resblock(64,128,3,1)(i) #32x32
i = resblock(128,256,3,2)(i) #16x16
i = resblock(256,512,3,2)(i) #8x8

i = BatchNormalization(axis=-1)(i)
i = relu(i)

i = AveragePooling2D(pool_size=(8,8),padding='valid')(i) #1x1

i = Conv2D(200,1)(i)
#i = Flatten()(i) # 128
i = GlobalAveragePooling2D()(i)

#i = Dense(10)(i)
i = Activation('softmax')(i)

model = Model(inputs=inp,outputs=i)

Instructions for updating:
Colocations handled automatically by placer.


In [ ]:
model.load_weights('./drive/My Drive/ChkPoint3/epochs:007-val_acc:0.326.hdf5')

In [ ]:
#LOAD IMAGES

import numpy as np
from PIL import Image
np.random.seed(1337)  # for reproducibility

#Keras
from keras.models import Sequential
from keras.layers.core import Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization 
from keras.utils import np_utils
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
# from keras.utils.visualize_util import plot
import h5py

import os
import numpy as np
from PIL import Image


def get_annotations_map():
  
  valAnnotationsPath = './tiny-imagenet-200/val/val_annotations.txt'
  valAnnotationsFile = open(valAnnotationsPath, 'r')
  valAnnotationsContents = valAnnotationsFile.read()
  valAnnotations = {}
  for line in valAnnotationsContents.splitlines():
    pieces = line.strip().split()
    valAnnotations[pieces[0]] = pieces[1]
  return valAnnotations

def load_images(path,num_classes):
    #Load images
    
    print('Loading ' + str(num_classes) + ' classes')

    X_train=np.zeros([num_classes*500,3,64,64],dtype='uint8')
    y_train=np.zeros([num_classes*500], dtype='uint8')
    trainPath=path+'/train'

    print('loading training images...');

    i=0
    j=0
    annotations={}
    for sChild in os.listdir(trainPath):
        sChildPath = os.path.join(os.path.join(trainPath,sChild),'images')
        annotations[sChild]=j
        for c in os.listdir(sChildPath):
            X=np.array(Image.open(os.path.join(sChildPath,c)))
            if len(np.shape(X))==2:
                X_train[i]=np.array([X,X,X])
            else:
                X_train[i]=np.transpose(X,(2,0,1))
            y_train[i]=j
            i+=1
        j+=1
        if (j >= num_classes):
            break

    print('finished loading training images')

    val_annotations_map = get_annotations_map()

    X_test = np.zeros([num_classes*50,3,64,64],dtype='uint8')
    y_test = np.zeros([num_classes*50], dtype='uint8')


    print('loading test images...')

    i = 0
    testPath=path+'/val/images'
    for sChild in os.listdir(testPath):
        if val_annotations_map[sChild] in annotations.keys():
            sChildPath = os.path.join(testPath, sChild)
            X=np.array(Image.open(sChildPath))
            if len(np.shape(X))==2:
                X_test[i]=np.array([X,X,X])
            else:
                X_test[i]=np.transpose(X,(2,0,1))
            y_test[i]=annotations[val_annotations_map[sChild]]
            i+=1
        else:
            pass


    print('finished loading test images ',end="")
    print(i)

    return X_train,y_train,X_test,y_test

loss_functions = ['categorical_crossentropy','squared_hinge','hinge']
num_classes = 200
batch_size = 128
nb_epoch = 30

#Load images
path='./tiny-imagenet-200/'
X_train,y_train,X_test,y_test=load_images(path,num_classes)

X_train = X_train.transpose(0,3,1,2)
X_test = X_test.transpose(0,3,1,2)

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print('X_test shape:', X_test.shape)
print(X_test.shape[0], 'test samples')

num_samples=len(X_train)

# input image dimensions
num_channels , img_rows, img_cols = X_train.shape[1], X_train.shape[2], X_train.shape[3]

X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, num_classes)
Y_test = np_utils.to_categorical(y_test, num_classes)

X_train = X_train.transpose(0,3,1,2)
X_test = X_test.transpose(0,3,1,2)
print (X_train.shape)
print (X_test.shape)

Loading 200 classes
loading training images...
finished loading training images
loading test images...
finished loading test images 10000
X_train shape: (100000, 64, 3, 64)
100000 train samples
X_test shape: (10000, 64, 3, 64)
10000 test samples
(100000, 64, 64, 3)
(10000, 64, 64, 3)


In [ ]:
from keras.callbacks import ModelCheckpoint

filepath="./drive/My Drive/ChkPoint3/epochs:{epoch:03d}-val_acc:{val_acc:.3f}.hdf5"

checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

callbacks_list = [checkpoint]

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# W/O ImgAug

model.fit(X_train, Y_train,
          batch_size=128,
          epochs=10,
          validation_data=(X_test, Y_test),
          shuffle=True,
          callbacks=callbacks_list)

Instructions for updating:
Use tf.cast instead.
Train on 100000 samples, validate on 10000 samples
Epoch 1/10
100000/100000 [==============================] - 769s 8ms/step - loss: 2.4989 - acc: 0.3982 - val_loss: 2.7703 - val_acc: 0.3553

Epoch 00001: val_acc improved from -inf to 0.35530, saving model to ./drive/My Drive/ChkPoint3/epochs:001-val_acc:0.355.hdf5
Epoch 2/10
100000/100000 [==============================] - 761s 8ms/step - loss: 2.2490 - acc: 0.4492 - val_loss: 2.8449 - val_acc: 0.3418

Epoch 00002: val_acc did not improve from 0.35530
Epoch 3/10
100000/100000 [==============================] - 760s 8ms/step - loss: 2.0905 - acc: 0.4828 - val_loss: 2.6559 - val_acc: 0.3719

Epoch 00003: val_acc improved from 0.35530 to 0.37190, saving model to ./drive/My Drive/ChkPoint3/epochs:003-val_acc:0.372.hdf5
Epoch 4/10
100000/100000 [==============================] - 760s 8ms/step - loss: 1.9594 - acc: 0.5097 - val_loss: 2.8023 - val_acc: 0.3803

Epoch 00004: val_acc improved from

In [ ]:
from keras.callbacks import ModelCheckpoint

filepath="./drive/My Drive/ChkPoint3/epochs:{epoch:03d}-val_acc:{val_acc:.3f}.hdf5"

checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

callbacks_list = [checkpoint]

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

# ImgAug and start training

print('Using real-time data augmentation.')
# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
	featurewise_center=False,  # set input mean to 0 over the dataset
	samplewise_center=False,  # set each sample mean to 0
	featurewise_std_normalization=False,  # divide inputs by std of the dataset
	samplewise_std_normalization=False,  # divide each input by its std
	zca_whitening=False,  # apply ZCA whitening
	rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
	width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
	height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
	horizontal_flip=True,  # randomly flip images
	vertical_flip=False)  # randomly flip images

# Compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(X_train)

# Fit the model on the batches generated by datagen.flow().
model.fit_generator(datagen.flow(X_train, Y_train, batch_size=128),
					steps_per_epoch=X_train.shape[0] // batch_size,
					validation_data=(X_test, Y_test),
					epochs=50, verbose=1, max_q_size=100,
					callbacks=callbacks_list)

Using real-time data augmentation.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., steps_per_epoch=781, validation_data=(array([[[..., epochs=50, verbose=1, callbacks=[<keras.ca..., max_queue_size=100)`


Epoch 1/50
781/781 [==============================] - 770s 986ms/step - loss: 1.9749 - acc: 0.5070 - val_loss: 2.6723 - val_acc: 0.4070

Epoch 00001: val_acc improved from -inf to 0.40700, saving model to ./drive/My Drive/ChkPoint3/epochs:001-val_acc:0.407.hdf5
Epoch 2/50
781/781 [==============================] - 755s 967ms/step - loss: 1.8542 - acc: 0.5329 - val_loss: 2.8092 - val_acc: 0.3878

Epoch 00002: val_acc did not improve from 0.40700
Epoch 3/50
781/781 [==============================] - 756s 968ms/step - loss: 1.7855 - acc: 0.5461 - val_loss: 2.3871 - val_acc: 0.4413

Epoch 00003: val_acc improved from 0.40700 to 0.44130, saving model to ./drive/My Drive/ChkPoint3/epochs:003-val_acc:0.441.hdf5
Epoch 4/50
781/781 [==============================] - 756s 968ms/step - loss: 1.7386 - acc: 0.5570 - val_loss: 2.5297 - val_acc: 0.4209

Epoch 00004: val_acc did not improve from 0.44130
Epoch 5/50
781/781 [==============================] - 755s 967ms/step - loss: 1.6944 - acc: 0.5668 

KeyboardInterrupt: ignored

In [ ]:
from keras.models import load_model

model.save('./drive/My Drive/my_model1.h5')  # creates a HDF5 file 'my_model.h5'

In [ ]:
model.fit_generator(datagen.flow(X_train, Y_train, batch_size=128),
					steps_per_epoch=X_train.shape[0] // batch_size,
					validation_data=(X_test, Y_test),
					epochs=10, verbose=1, max_q_size=100,
					callbacks=callbacks_list)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., steps_per_epoch=781, validation_data=(array([[[..., epochs=10, verbose=1, callbacks=[<keras.ca..., max_queue_size=100)`
  """


Epoch 1/10
781/781 [==============================] - 756s 968ms/step - loss: 1.1328 - acc: 0.6918 - val_loss: 2.5347 - val_acc: 0.4718

Epoch 00001: val_acc did not improve from 0.49030
Epoch 2/10
  4/781 [..............................] - ETA: 10:01 - loss: 1.0406 - acc: 0.7168

In [ ]:
from keras.models import load_model
# returns a compiled model
# identical to the previous one
model = load_model('./drive/My Drive/my_model1.h5')

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [ ]:
from keras.callbacks import ModelCheckpoint

filepath="./drive/My Drive/ChkPoint3/epochs:{epoch:03d}-val_acc:{val_acc:.3f}.hdf5"

checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')

callbacks_list = [checkpoint]


# ImgAug and start training

print('Using real-time data augmentation.')
# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
	featurewise_center=False,  # set input mean to 0 over the dataset
	samplewise_center=False,  # set each sample mean to 0
	featurewise_std_normalization=False,  # divide inputs by std of the dataset
	samplewise_std_normalization=False,  # divide each input by its std
	zca_whitening=False,  # apply ZCA whitening
	rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
	width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
	height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
	horizontal_flip=True,  # randomly flip images
	vertical_flip=False)  # randomly flip images

# Compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(X_train)

# Fit the model on the batches generated by datagen.flow().
model.fit_generator(datagen.flow(X_train, Y_train, batch_size=128),
					steps_per_epoch=X_train.shape[0] // batch_size,
					validation_data=(X_test, Y_test),
					epochs=50, verbose=1, max_q_size=100,
					callbacks=callbacks_list)

Using real-time data augmentation.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:35: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<keras_pre..., steps_per_epoch=781, validation_data=(array([[[..., epochs=50, verbose=1, callbacks=[<keras.ca..., max_queue_size=100)`


Epoch 1/50
781/781 [==============================] - 828s 1s/step - loss: 1.1306 - acc: 0.6930 - val_loss: 2.6151 - val_acc: 0.4628

Epoch 00001: val_acc improved from -inf to 0.46280, saving model to ./drive/My Drive/ChkPoint3/epochs:001-val_acc:0.463.hdf5
Epoch 2/50
781/781 [==============================] - 816s 1s/step - loss: 1.1027 - acc: 0.7001 - val_loss: 2.6139 - val_acc: 0.4737

Epoch 00002: val_acc improved from 0.46280 to 0.47370, saving model to ./drive/My Drive/ChkPoint3/epochs:002-val_acc:0.474.hdf5
Epoch 3/50
781/781 [==============================] - 815s 1s/step - loss: 1.0733 - acc: 0.7063 - val_loss: 2.5028 - val_acc: 0.4832

Epoch 00003: val_acc improved from 0.47370 to 0.48320, saving model to ./drive/My Drive/ChkPoint3/epochs:003-val_acc:0.483.hdf5
Epoch 4/50
781/781 [==============================] - 815s 1s/step - loss: 1.0553 - acc: 0.7109 - val_loss: 2.4660 - val_acc: 0.4881

Epoch 00004: val_acc improved from 0.48320 to 0.48810, saving model to ./drive/My D

KeyboardInterrupt: ignored